In [2]:
# #Install Packages
# !pip install faiss-cpu
# !pip install sentence_transformers

In [3]:
import pandas as pd

pd.set_option('display.max_colwidth', 100)

In [4]:
df = pd.read_csv('./sample_text.csv')
df.shape

(8, 2)

In [5]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [6]:
print(df.text)

0                                          Meditation and yoga can improve mental health
1                       Fruits, whole grains and vegetables helps control blood pressure
2                                      These are the latest fashion trends for this week
3                                      Vibrant color jeans for male are becoming a trend
4                                                     The concert starts at 7 PM tonight
5                        Navaratri dandiya program at Expo center in Mumbai this october
6                                      Exciting vacation destinations for your next trip
7    Maldives and Srilanka are gaining popularity in terms of low budget vacation places
Name: text, dtype: object


In [7]:
from sentence_transformers import SentenceTransformer

encoder = SentenceTransformer("avsolatorio/GIST-Embedding-v0")
vectors = encoder.encode(df.text)
vectors.shape

(8, 768)

In [8]:
encoder.max_seq_length

512

In [9]:
dim = vectors.shape[1]
dim

768

In [10]:
import faiss
# print(help(faiss.IndexFlatL2))
index = faiss.IndexFlatL2(dim)
index

<faiss.swigfaiss.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000001921F348510> >

In [11]:
index.add(vectors)

In [12]:
search_query = "I want to go to the beach"
vec = encoder.encode(sentences=search_query)
vec.shape

(768,)

In [13]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape

(1, 768)

In [14]:
svec

array([[-3.82514223e-02, -1.53553355e-02, -7.76748406e-03,
        -3.42873223e-02,  3.00935581e-02, -8.44205264e-03,
         3.42817046e-02,  3.92005518e-02, -3.20673659e-02,
        -5.71211688e-02,  1.05128725e-04,  3.05951992e-03,
        -2.32191626e-02,  1.75354760e-02,  2.04758141e-02,
         1.79510806e-02,  3.06967236e-02,  5.22467587e-03,
         1.82236806e-02, -7.29599735e-03,  1.93084218e-02,
         6.30076975e-02, -4.49153185e-02,  3.49803492e-02,
         4.95070703e-02, -1.42291496e-02,  7.72644207e-02,
         2.52888128e-02, -6.47956505e-02, -3.86661440e-02,
        -1.06603128e-03, -2.10870663e-03, -3.06838937e-02,
         3.89753170e-02, -2.13300548e-02, -1.54706324e-02,
        -9.40539583e-04, -3.77519242e-02,  6.67015556e-04,
        -1.56294908e-02, -2.23657247e-02, -2.80875266e-02,
         2.45730653e-02, -1.70639511e-02, -1.26081547e-02,
         3.96172255e-02,  1.18935863e-02,  2.11555511e-03,
        -1.74774285e-02, -2.37401910e-02, -4.44907248e-0

In [15]:
distances, indexes = index.search(svec, k=2)

In [16]:
indexes

array([[6, 7]], dtype=int64)

In [17]:
df.loc[indexes[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [18]:
df.loc[[6,7]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


In [19]:
search_query

'I want to go to the beach'

In [20]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


### faiss-gpu

In [21]:
import faiss

# For GPU
print(faiss.get_num_gpus())

1


In [22]:
# Inisialisasi resources GPU
resources = faiss.StandardGpuResources()

In [23]:
index = faiss.IndexFlatL2(dim)

# One GPU
gpu_index_1 = faiss.index_cpu_to_gpu(provider=resources, device=0, index=index)
print(gpu_index_1)

# All GPU
gpu_index_multi = faiss.index_cpu_to_all_gpus(index=index)
gpu_index_multi

<faiss.swigfaiss.GpuIndexFlat; proxy of <Swig Object of type 'faiss::gpu::GpuIndexFlat *' at 0x000001921F4D2850> >


<faiss.swigfaiss.GpuIndexFlat; proxy of <Swig Object of type 'faiss::gpu::GpuIndexFlat *' at 0x000001921F326280> >

In [24]:
gpu_index_multi.add(vectors)

In [25]:
distances, indexes = gpu_index_multi.search(svec, k=2)

In [26]:
indexes

array([[6, 7]], dtype=int64)

In [27]:
df.loc[indexes[0]]

,text,category
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel
